# Testes básicos de acesso e parsing

In [1]:
import requests

api_url = "https://jsonplaceholder.typicode.com/todos/1"
response = requests.get(api_url)
response.json()



{'userId': 1, 'id': 1, 'title': 'delectus aut autem', 'completed': False}

In [2]:
api_url = "https://pesquisa.apps.tcu.gov.br/rest/publico/base/jurisprudencia-selecionada/documentosResumidos?termo=t%C3%A9cnica%20e%20pre%C3%A7o&filtro=&ordenacao=score%20desc,%20COLEGIADO%20asc,%20ANOACORDAO%20desc,%20NUMACORDAO%20desc&quantidade=20&inicio=0&sinonimos=true"
response = requests.get(api_url)
response.json()


{'quantidadeEncontrada': 187,
 'inicio': 0,
 'documentos': [{'KEY': 'JURISPRUDENCIA-SELECIONADA-34240',
   'FAVORITO': False,
   'DTATUALIZACAO': '20170512',
   'NUMACORDAO': '1782',
   'ANOACORDAO': '2004',
   'COLEGIADO': 'Plenário',
   'AREA': 'Licitação',
   'TEMA': 'Parcelamento do objeto',
   'SUBTEMA': 'Obrigatoriedade',
   'ASSUNTO': '',
   'ENUNCIADO': 'SÚMULA TCU 247: É obrigatória a admissão da adjudicação por item e não por <em>preço</em> global, nos editais das licitações para a contratação de obras, serviços, compras e alienações, cujo objeto seja divisível, desde que não haja prejuízo para o conjunto ou complexo ou perda de economia de escala, tendo em vista o objetivo de propiciar a ampla participação de licitantes que, embora não dispondo de capacidade para a execução, fornecimento ou aquisição da totalidade do objeto, possam fazê-lo com relação a itens ou unidades autônomas, devendo as exigências de habilitação adequar-se a essa divisibilidade.',
   'NUMSUMULA': '',
 

In [10]:
[int(doc['KEY'].split('-')[-1]) if doc['KEY'].startswith('JURISPRUDENCIA-SELECIONADA-') else 0 for doc in response.json()['documentos']]

[34240,
 20956,
 68513,
 51881,
 39213,
 21229,
 21285,
 17125,
 18324,
 21159,
 20870,
 20969,
 20970,
 20971,
 20869,
 34126,
 31601,
 20871,
 21072,
 34233]

In [12]:
import urllib.parse

query = 'técnica e preço'
quoted_query = urllib.parse.quote(query)
query + ' --> ' + quoted_query

'técnica e preço --> t%C3%A9cnica%20e%20pre%C3%A7o'

In [13]:
api_url = f"https://pesquisa.apps.tcu.gov.br/rest/publico/base/jurisprudencia-selecionada/documentosResumidos?termo={quoted_query}&filtro=&ordenacao=score%20desc,%20COLEGIADO%20asc,%20ANOACORDAO%20desc,%20NUMACORDAO%20desc&quantidade=20&inicio=0&sinonimos=true"
api_url

'https://pesquisa.apps.tcu.gov.br/rest/publico/base/jurisprudencia-selecionada/documentosResumidos?termo=t%C3%A9cnica%20e%20pre%C3%A7o&filtro=&ordenacao=score%20desc,%20COLEGIADO%20asc,%20ANOACORDAO%20desc,%20NUMACORDAO%20desc&quantidade=20&inicio=0&sinonimos=true'

# Carga das queries e execução da busca

In [14]:
import pandas as pd

df_queries = pd.read_csv('../../data/juris_tcu/query.csv')
df_queries

,ID,TEXT,SOURCE
0,1,técnica e preço,search log
1,2,restos a pagar,search log
2,3,aditivo a contrato,search log
3,4,adesão a ata de registro de preços,search log
4,5,sobrepreço e superfaturamento,search log
...,...,...,...
145,146,É possível a inclusão de serviços já previstos...,LLM
146,147,O princípio da legalidade estrita pode ser afa...,LLM
147,148,Quais fontes devem ser prioritárias na pesquis...,LLM
148,149,Devem ser aceitas nas contratações de software...,LLM


In [20]:
result = {}
for id, query in list(df_queries[['ID', 'TEXT']].values):
    quoted_query = urllib.parse.quote(query)
    api_url = f"https://pesquisa.apps.tcu.gov.br/rest/publico/base/jurisprudencia-selecionada/documentosResumidos?termo={quoted_query}&filtro=&ordenacao=score%20desc,%20COLEGIADO%20asc,%20ANOACORDAO%20desc,%20NUMACORDAO%20desc&quantidade=20&inicio=0&sinonimos=true"
    print('Query', id, '-', query+  ':')
    response = requests.get(api_url)
    result[id] = [int(doc['KEY'].split('-')[-1]) if doc['KEY'].startswith('JURISPRUDENCIA-SELECIONADA-') else 0 for doc in response.json()['documentos']]
    print('\t', result[id])


Query 1 - técnica e preço:
	 [34240, 20956, 68513, 51881, 39213, 21229, 21285, 17125, 18324, 21159, 20870, 20969, 20970, 20971, 20869, 34126, 31601, 20871, 21072, 34233]
Query 2 - restos a pagar:
	 [53678, 39200, 112346, 47053, 56445, 18430, 103476, 32869, 77959, 18105, 19084, 17289, 19340, 31437, 76612, 18432, 18452]
Query 3 - aditivo a contrato:
	 [17089, 108415, 10089, 11611, 11612, 59295, 59296, 104455, 81314, 81554, 59318, 59317, 34116, 18416, 17816, 49716, 139160, 31084, 15197, 31120]
Query 4 - adesão a ata de registro de preços:
	 [137717, 20949, 15092, 15372, 20839, 52912, 4230, 15178, 147727, 62306, 50537, 16895, 17195, 70413, 20860, 20841, 41295, 49694, 20840, 42169]
Query 5 - sobrepreço e superfaturamento:
	 [17816, 148920, 23515, 20524, 17813, 16932, 21171, 32590, 21227, 100548, 77235, 21172, 86727, 9527, 18393, 20523, 53003, 16262, 21793, 22908]
Query 6 - restrição a competitividade:
	 [34918, 18286, 17165, 17916, 13109, 18259, 25442, 14527, 16409, 17372, 23050, 59438, 170

In [32]:
with open('../../data/search/juris_tcu/run_tcu_search.csv', 'w') as run:
    run.write('QUERY_ID,DOC_ID,RANK,ENGINE\n')
    for query_id, docs in result.items():
        for rank, doc in enumerate(docs):
            run.write(f'{query_id},{doc},{rank + 1},TCU_Search\n')

In [33]:
df_run_tcu_search = pd.read_csv('../../data/search/juris_tcu/run_tcu_search.csv')
df_run_tcu_search

,QUERY_ID,DOC_ID,RANK,ENGINE
0,1,34240,1,TCU_Search
1,1,20956,2,TCU_Search
2,1,68513,3,TCU_Search
3,1,51881,4,TCU_Search
4,1,39213,5,TCU_Search
...,...,...,...,...
1142,145,27209,1,TCU_Search
1143,145,86727,2,TCU_Search
1144,145,43507,3,TCU_Search
1145,145,61975,4,TCU_Search


In [31]:
df_run_juris_tcu = pd.read_csv('../../data/juris_tcu/qrel.csv')
df_run_juris_tcu

,QUERY_ID,DOC_ID,SCORE,ENGINE,RANK
0,1,21064,3,(BM25|STS)+Reranker+LLM,1
1,1,42166,3,(BM25|STS)+Reranker+LLM,2
2,1,13702,3,(BM25|STS)+Reranker+LLM,3
3,1,20592,3,(BM25|STS)+Reranker+LLM,4
4,1,15740,3,(BM25|STS)+Reranker+LLM,5
...,...,...,...,...,...
2245,150,117370,1,BM25+LLM,190
2246,150,14904,0,BM25+LLM,185
2247,150,34204,0,BM25+LLM,243
2248,150,70288,0,BM25+LLM,328


In [48]:
df_bm25 = pd.read_csv('../../data/search/juris_tcu/run_bm25.csv')
df_bm25_reranker = pd.read_csv('../../data/search/juris_tcu/run_bm25_reranker.csv')
df_sts = pd.read_csv('../../data/search/juris_tcu/run_sts.csv')
df_sts_reranker = pd.read_csv('../../data/search/juris_tcu/run_sts_reranker.csv')
df_join_bm25_sts_reranker = pd.read_csv('../../data/search/juris_tcu/run_join_bm25_sts_reranker.csv')

In [68]:
df_compare_tcu_juris = df_run_tcu_search.merge(df_run_juris_tcu, how='inner', on=['QUERY_ID', 'DOC_ID'], suffixes=('_TCU', '_JURIS')).groupby('QUERY_ID').agg({'DOC_ID': 'count', 'SCORE': 'mean'})
df_compare_tcu_juris.columns = ['TCU_JURIS_DOCS', 'TCU_JURIS_AVG_SCORE']
df_compare_tcu_juris

,TCU_JURIS_DOCS,TCU_JURIS_AVG_SCORE
QUERY_ID,,
1,1,3.000000
2,6,2.333333
3,1,3.000000
4,3,3.000000
5,3,3.000000
...,...,...
122,1,3.000000
137,1,3.000000
140,2,3.000000


In [69]:
df = df_run_tcu_search.merge(df_bm25, how='inner', on=['QUERY_ID', 'DOC_ID'], suffixes=('_TCU', '_JURIS')).groupby('QUERY_ID').agg({'DOC_ID': 'count'})
df.columns = ['BM25_DOCS']
df_compare_tcu_juris = df_compare_tcu_juris.merge(df, on='QUERY_ID', how='outer').fillna(0).sort_index()
df_compare_tcu_juris

,TCU_JURIS_DOCS,TCU_JURIS_AVG_SCORE,BM25_DOCS
QUERY_ID,,,
1,1.0,3.000000,18
2,6.0,2.333333,16
3,1.0,3.000000,16
4,3.0,3.000000,20
5,3.0,3.000000,20
...,...,...,...
122,1.0,3.000000,1
137,1.0,3.000000,1
140,2.0,3.000000,2


In [70]:
df = df_run_tcu_search.merge(df_bm25_reranker, how='inner', on=['QUERY_ID', 'DOC_ID'], suffixes=('_TCU', '_JURIS')).groupby('QUERY_ID').agg({'DOC_ID': 'count'})
df.columns = ['BM25_RR_DOCS']
df_compare_tcu_juris = df_compare_tcu_juris.merge(df, on='QUERY_ID', how='outer').fillna(0).sort_index()
df_compare_tcu_juris

,TCU_JURIS_DOCS,TCU_JURIS_AVG_SCORE,BM25_DOCS,BM25_RR_DOCS
QUERY_ID,,,,
1,1.0,3.000000,18,13.0
2,6.0,2.333333,16,16.0
3,1.0,3.000000,16,7.0
4,3.0,3.000000,20,20.0
5,3.0,3.000000,20,20.0
...,...,...,...,...
122,1.0,3.000000,1,1.0
137,1.0,3.000000,1,1.0
140,2.0,3.000000,2,2.0


In [71]:
df = df_run_tcu_search.merge(df_sts, how='inner', on=['QUERY_ID', 'DOC_ID'], suffixes=('_TCU', '_JURIS')).groupby('QUERY_ID').agg({'DOC_ID': 'count'})
df.columns = ['STS_DOCS']
df_compare_tcu_juris = df_compare_tcu_juris.merge(df, on='QUERY_ID', how='outer').fillna(0).sort_index()
df_compare_tcu_juris

,TCU_JURIS_DOCS,TCU_JURIS_AVG_SCORE,BM25_DOCS,BM25_RR_DOCS,STS_DOCS
QUERY_ID,,,,,
1,1.0,3.000000,18,13.0,16.0
2,6.0,2.333333,16,16.0,5.0
3,1.0,3.000000,16,7.0,6.0
4,3.0,3.000000,20,20.0,7.0
5,3.0,3.000000,20,20.0,12.0
...,...,...,...,...,...
122,1.0,3.000000,1,1.0,1.0
137,1.0,3.000000,1,1.0,1.0
140,2.0,3.000000,2,2.0,2.0


In [72]:
df = df_run_tcu_search.merge(df_sts_reranker, how='inner', on=['QUERY_ID', 'DOC_ID'], suffixes=('_TCU', '_JURIS')).groupby('QUERY_ID').agg({'DOC_ID': 'count'})
df.columns = ['STS_RR_DOCS']
df_compare_tcu_juris = df_compare_tcu_juris.merge(df, on='QUERY_ID', how='outer').fillna(0).sort_index()
df_compare_tcu_juris

,TCU_JURIS_DOCS,TCU_JURIS_AVG_SCORE,BM25_DOCS,BM25_RR_DOCS,STS_DOCS,STS_RR_DOCS
QUERY_ID,,,,,,
1,1.0,3.000000,18,13.0,16.0,15.0
2,6.0,2.333333,16,16.0,5.0,1.0
3,1.0,3.000000,16,7.0,6.0,6.0
4,3.0,3.000000,20,20.0,7.0,6.0
5,3.0,3.000000,20,20.0,12.0,11.0
...,...,...,...,...,...,...
122,1.0,3.000000,1,1.0,1.0,1.0
137,1.0,3.000000,1,1.0,1.0,1.0
140,2.0,3.000000,2,2.0,2.0,2.0


In [73]:
df = df_run_tcu_search.merge(df_join_bm25_sts_reranker, how='inner', on=['QUERY_ID', 'DOC_ID'], suffixes=('_TCU', '_JURIS')).groupby('QUERY_ID').agg({'DOC_ID': 'count'})
df.columns = ['BM25_STS_RR_DOCS']
df_compare_tcu_juris = df_compare_tcu_juris.merge(df, on='QUERY_ID', how='outer').fillna(0).sort_index()
df_compare_tcu_juris

,TCU_JURIS_DOCS,TCU_JURIS_AVG_SCORE,BM25_DOCS,BM25_RR_DOCS,STS_DOCS,STS_RR_DOCS,BM25_STS_RR_DOCS
QUERY_ID,,,,,,,
1,1.0,3.000000,18,13.0,16.0,15.0,17
2,6.0,2.333333,16,16.0,5.0,1.0,16
3,1.0,3.000000,16,7.0,6.0,6.0,8
4,3.0,3.000000,20,20.0,7.0,6.0,20
5,3.0,3.000000,20,20.0,12.0,11.0,20
...,...,...,...,...,...,...,...
122,1.0,3.000000,1,1.0,1.0,1.0,1
137,1.0,3.000000,1,1.0,1.0,1.0,1
140,2.0,3.000000,2,2.0,2.0,2.0,2


In [82]:
df = df_run_tcu_search.groupby('QUERY_ID').agg({'DOC_ID': 'count'})
df.columns = ['TCU_SEARCH_DOCS']
df_compare_tcu_juris = df_compare_tcu_juris.merge(df, on='QUERY_ID', how='outer').fillna(0).sort_index()
df_compare_tcu_juris

,TCU_JURIS_DOCS,TCU_JURIS_AVG_SCORE,BM25_DOCS,BM25_RR_DOCS,STS_DOCS,STS_RR_DOCS,BM25_STS_RR_DOCS,TCU_SEARCH_DOCS
QUERY_ID,,,,,,,,
1,1.0,3.000000,18,13.0,16.0,15.0,17,20
2,6.0,2.333333,16,16.0,5.0,1.0,16,17
3,1.0,3.000000,16,7.0,6.0,6.0,8,20
4,3.0,3.000000,20,20.0,7.0,6.0,20,20
5,3.0,3.000000,20,20.0,12.0,11.0,20,20
...,...,...,...,...,...,...,...,...
122,1.0,3.000000,1,1.0,1.0,1.0,1,1
137,1.0,3.000000,1,1.0,1.0,1.0,1,1
140,2.0,3.000000,2,2.0,2.0,2.0,2,2


In [92]:
#del df_compare_tcu_juris['TCU_JURIS_AVG_SCORE']
df_compare_tcu_juris = df_compare_tcu_juris.astype('int32')

In [93]:
df_compare_tcu_juris.sum().sort_values()

TCU_JURIS_DOCS       385
STS_RR_DOCS          511
STS_DOCS             627
BM25_RR_DOCS         927
BM25_STS_RR_DOCS     969
BM25_DOCS           1028
TCU_SEARCH_DOCS     1147
dtype: int64

In [94]:
df_compare_tcu_juris.to_csv('../../data/search/juris_tcu/compare_search_tcu.csv')